### Imports

In [263]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objects as go
from shapely.wkt import loads
import numpy as np
from datetime import datetime

#### Processing datasets function

In [264]:
#process trees csv
def process_data(file_path):
    """Process CSV file and return cleaned dataframe."""
    try:
        df = pd.read_csv(
            file_path,
            sep=';',
            encoding='utf-8-sig',
            on_bad_lines='skip',
            engine='python'
        )
        # Convert coordinates to float
        df['LAT'] = pd.to_numeric(df['LAT'], errors='coerce')
        df['LNG'] = pd.to_numeric(df['LNG'], errors='coerce')
        df = df.dropna(subset=['LAT', 'LNG'])

        return df
    except Exception as e:
        print(f"Error processing trees data: {e}")
        return pd.DataFrame()


#process the zipcodes file
def process_zipcodes(file_path):
    """Process CSV file and return dataframe."""
    try:
        df = pd.read_csv(
            file_path,
            sep=';',
            encoding='utf-8-sig'
        )
        return df
    except Exception as e:
        print(f"Error processing zipcodes data: {e}")
        return pd.DataFrame()

#### Map creation function

In [265]:
#create hexbin map
def create_map(trees_df, zipcodes_df, selected_zip=None, selected_species=None):
    """Generate a map with zip code polygons and tree locations."""
    fig = go.Figure()

    #set default zoom center for Amsterdam
    center_lat = 52.3676
    center_lng = 4.9041
    zoom_level = 11  #default zoom level for Amsterdam

    #compute and add zip code polygons
    for _, row in zipcodes_df.iterrows():
        polygons = []

        #load 'WKT_LNG_LAT' and 'WKT_LAT_LNG'
        try:
            polygons.append(loads(row['WKT_LNG_LAT']))
        except:
            pass

        try:
            polygons.append(loads(row['WKT_LAT_LNG']))
        except:
            pass

        #loop through polygons and add to map
        for polygon in polygons:
            if polygon.geom_type == 'MultiPolygon':
                for single_polygon in polygon.geoms:
                    x, y = single_polygon.exterior.xy
                    opacity = 0.8 if row['Postcode4'] == selected_zip else 0.3
                    fig.add_trace(go.Scattermapbox(
                        lon=list(x),
                        lat=list(y),
                        mode='lines',
                        fill='toself',
                        fillcolor='rgba(128, 128, 128, 0.3)',
                        line=dict(color='blue', width=1),
                        opacity=opacity,
                        name=f"Zipcode {row['Postcode4']}",
                        showlegend=False
                    ))
            elif polygon.geom_type == 'Polygon':
                x, y = polygon.exterior.xy
                opacity = 0.8 if row['Postcode4'] == selected_zip else 0.3
                fig.add_trace(go.Scattermapbox(
                    lon=list(x),
                    lat=list(y),
                    mode='lines',
                    fill='toself',
                    fillcolor='rgba(128, 128, 128, 0.3)',
                    line=dict(color='blue', width=1),
                    opacity=opacity,
                    name=f"Zipcode {row['Postcode4']}",
                    showlegend=False
                ))

                #when the selected zip code is found, use its center coordinates
                if row['Postcode4'] == selected_zip:
                    #compute bounding box of the polygon
                    min_lon, min_lat, max_lon, max_lat = polygon.bounds
                    center_lat = (min_lat + max_lat) / 2
                    center_lng = (min_lon + max_lon) / 2

                    #set zoom level based on the size of the bounding box
                    zoom_level = 14 #zoom in for selected zip code
                    #compute aspect ratio to fine-tune zoom level
                    aspect_ratio = (max_lat - min_lat) / (max_lon - min_lon)
                    zoom_level = 15 if aspect_ratio < 0.5 else zoom_level

    #filter trees
    filtered_df = trees_df.copy()
    if selected_species:
        filtered_df = filtered_df[filtered_df['Soortnaam'].isin(selected_species)]

    #add trees as density points on the map
    if not filtered_df.empty:
        fig.add_trace(go.Scattermapbox(
            lon=filtered_df['LNG'],
            lat=filtered_df['LAT'],
            mode='markers',
            marker=dict(size=5, color='green', opacity=0.6),
            text=filtered_df['Soortnaam'],
            name='Trees'
        ))

    #update layout
    fig.update_layout(
        mapbox=dict(
            style='carto-positron',
            center=dict(lat=center_lat, lon=center_lng),
            zoom=zoom_level
        ),
        margin=dict(l=0, r=0, t=30, b=0),
        height=600
    )

    return fig

#### Load and process the data

In [266]:
#load and process the data
trees_df = process_data("/Users/lauravochita/PycharmProjects/pythonDSP/DSP_dashboard/trees.csv")
zipcodes_df = process_zipcodes("/Users/lauravochita/PycharmProjects/pythonDSP/DSP_dashboard/zipcodes.csv")

#### Unique values for dropdowns

In [267]:
#get unique values for dropdowns
unique_zipcodes = sorted(zipcodes_df['Postcode4'].unique())
unique_species = trees_df[trees_df['Soortnaam'].notna()]['Soortnaam'].unique()

####  Dash app

In [268]:
#initialize Dash app
app = dash.Dash(__name__, assets_folder='/Users/lauravochita/PycharmProjects/pythonDSP/DSP_dashboard/assets')

In [269]:
#set layout of Dash app
app.layout = html.Div([
    #header Section
    html.Div([
        html.Img(
            src= '/assets/Logo_of_Gemeente_Amsterdam.png',
            style={
                'height': '50px',
                'marginRight': '15px',
                'verticalAlign': 'middle'
            }
        ),
        html.H1(
            "Amsterdam Particulate Matter Simulator",
            style={
                'display': 'inline-block',
                'verticalAlign': 'middle',
                'color': 'black',
                'fontSize': '36px',
                'fontWeight': 'bold',
                'fontFamily': 'Verdana',
                'margin': '0'
            }
        )
    ], style={
        'display': 'flex',
        'alignItems': 'center',
        'justifyContent': 'center',
        'marginBottom': '30px',
        'marginTop': '0',
        'padding': '10px 20px',
        'backgroundColor': '#f9f9f9',
        'boxShadow': '0 2px 4px rgba(0, 0, 0, 0.1)'
    }),

    #filters Section
    html.Div([
        #filters row 1 (Zip Code and Tree Species Selection dropdowns)
        html.Div([
            html.Div([
                html.Label("Select Zip Code:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='zipcode-dropdown',
                    options=[{'label': str(code), 'value': code} for code in unique_zipcodes],
                    placeholder="Select a zip code...",
                    style={'width': '100%'}
                )
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 10px'}),

            html.Div([
                html.Label("Select Tree Species:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='tree-dropdown',
                    options=[{'label': name, 'value': name} for name in unique_species],
                    multi=True,
                    placeholder="Select tree(s)...",
                    style={'width': '100%'}
                )
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 10px'})
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '20px'}),

        #filters row 2 (Add Tree(s) for Simulation and Number of Simulations dropdowns)
        html.Div([
            html.Div([
                html.Label("Add Tree(s) for Simulation:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='add-trees-dropdown',
                    options=[{'label': name, 'value': name} for name in trees_df['Soortnaam'].unique()],
                    multi=True,
                    placeholder="Select tree(s) to add...",
                    style={'width': '100%'}
                )
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 10px'}),

            html.Div([
                html.Label("Number of Simulations:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='simulation-count-dropdown',
                    options=[
                        {'label': '5 simulations', 'value': 5},
                        {'label': '10 simulations', 'value': 10},
                        {'label': '20 simulations', 'value': 20},
                        {'label': '25 simulations', 'value': 25}
                    ],
                    placeholder="Select number of simulations...",
                    style={'width': '100%'}
                )
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 10px'})
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '20px'}),

        #date Range Picker
        html.Div([
            html.Label("Select Date Range:", style={'fontWeight': 'bold'}),
            dcc.DatePickerRange(
                id='date-range-picker',
                min_date_allowed=datetime(2024, 6, 6).date(),
                initial_visible_month=datetime(2024, 6, 6).date(),
                style={'width': '100%'}
            )
        ], style={'marginBottom': '30px', 'padding': '0 10px'}),

        #simulate button and error messages
        html.Div([
            html.Button(
                "SIMULATE",
                id="simulate-btn",
                n_clicks=0,
                style={
                    'fontSize': '16px',
                    'fontWeight': 'bold',
                    'color': '#ffffff',
                    'backgroundColor': '#ed0000',
                    'border': 'none',
                    'borderRadius': '5px',
                    'padding': '10px 20px',
                    'cursor': 'pointer',
                    'boxShadow': '0 2px 4px rgba(0, 0, 0, 0.2)',
                    'width': '200px'
                }
            ),
            html.Div(
                id='simulation-error-message',
                style={'color': 'red', 'marginTop': '10px'}
            )
        ], style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '15px'}),

    ], style={
        'backgroundColor': '#f0f0f0',
        'borderRadius': '8px',
        'padding': '20px',
        'marginBottom': '30px',
        'boxShadow': '0 2px 4px rgba(0, 0, 0, 0.1)'
    }),

    #main content section
    html.Div([
        #map on the left
        html.Div([
            dcc.Graph(
                id='map-plot',
                figure=create_map(trees_df, zipcodes_df),
                style={
                    'width': '100%',
                    'height': '500px'
                }
            )
        ], style={
            'width': '60%',
            'display': 'inline-block',
            'padding': '0 10px'
        }),

        #simulation plot placeholder on the right
        html.Div([
            html.Div(
                id='simulation-plot-container',
                children=html.H4("graph after pressing SIMULATE button",
                                 style={'textAlign': 'center', 'color': 'black'}),
                style={'height': '100%'}
            )
        ], style={
            'width': '40%',
            'display': 'inline-block',
            'padding': '0 10px',
            'textAlign': 'center',
            'backgroundColor': '#f9f9f9',
            'border': '1px solid #e0e0e0',
            'borderRadius': '8px',
            'height': '500px'
        })
    ], style={
        'display': 'flex',
        'justifyContent': 'space-between',
        'marginBottom': '30px'
    }),
], style={'fontFamily': 'Verdana'})


#### Callbacks

In [270]:
#callback to update the map
@app.callback(
    Output('map-plot', 'figure'),
    [Input('zipcode-dropdown', 'value'),
     Input('tree-dropdown', 'value')]
)
def update_map(selected_zip, selected_species):
    return create_map(trees_df, zipcodes_df, selected_zip, selected_species)

#callback for enabling/disabling the simulation button
@app.callback(
    [Output('simulate-btn', 'disabled'),
     Output('simulation-error-message', 'children')],
    [Input('simulation-count-dropdown', 'value'),
     Input('zipcode-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date'),
     Input('add-trees-dropdown', 'value')]
)
def enable_button(sim_count, zipcode, start_date, end_date, trees):
    #check each field is filled in and return the first error message found
    if not zipcode:
        return True, "Please select a zip code."
    if not trees:
        return True, "Please select trees to add."
    if not start_date or not end_date:
        return True, "Please select a valid date range."
    if not sim_count:
        return True, "Please select the number of simulations."

    #if no errors, enable the button and clear the error messages
    return False, ""

#callback for simulation and plot display
@app.callback(
    Output("simulation-plot-container", "children"),
    [
        Input("simulate-btn", "n_clicks")
    ],
    [
        State("date-range-picker", "start_date"),
        State("date-range-picker", "end_date"),
        State("zipcode-dropdown", "value"),
        State("add-trees-dropdown", "value"),
        State("simulation-count-dropdown", "value")
    ]
)
def run_simulation(n_clicks, start_date, end_date, zip_code, tree_type, sim_count):
    if n_clicks > 0:
        #display the pre-generated simulation plot
        return html.Div(
            children=[
                html.Img(
                    src="/assets/simulation-plot.jpeg",
                    style={
                        "width": "100%",  #make it fill the space
                        "height": "auto",  #maintain aspect ratio
                        "maxHeight": "90%",  #allow it to be 90% of the container's height
                        "objectFit": "contain",  #enure the image fits well without stretching
                        "borderRadius": "8px",  #rounded corners
                        "display": "block",
                        "margin": "0 auto"  #center the image horizontally
                    }
                )
            ],
            style={
                'display': 'flex',
                'alignItems': 'center',  #vertically center
                'justifyContent': 'center',  #horizontally center
                'textAlign': 'center',  #ensure text is centered within the container
                'padding': '20px',
                'height': '500px'  #ensure the container has a fixed height
            }
        )
    return html.Div(
        "Simulation plot will appear here after running.",
        style={
            'textAlign': 'center', 
            'color': 'black',
            'fontSize': '18px',
            'fontWeight': 'bold',
            'padding': '20px'
        }
    )
if __name__ == '__main__':
    app.run_server(debug=True)